# Day 5: Print Queue 
```
Satisfied with their search on Ceres, the squadron of scholars suggests subsequently scanning the stationery stacks of sub-basement 17.

The North Pole printing department is busier than ever this close to Christmas, and while The Historians continue their search of this historically significant facility, an Elf operating a very familiar printer beckons you over.

The Elf must recognize you, because they waste no time explaining that the new sleigh launch safety manual updates won't print correctly. Failure to update the safety manuals would be dire indeed, so you offer your services.

Safety protocols clearly indicate that new pages for the safety manuals must be printed in a very specific order. The notation X|Y means that if both page number X and page number Y are to be produced as part of an update, page number X must be printed at some point before page number Y.

The Elf has for you both the page ordering rules and the pages to produce in each update (your puzzle input), but can't figure out whether each update has the pages in the right order.

For example:

47|53
97|13
97|61
97|47
75|29
61|13
75|53
29|13
97|29
53|29
61|53
97|53
61|29
47|13
75|47
97|75
47|61
75|61
47|29
75|13
53|13

75,47,61,53,29
97,61,53,29,13
75,29,13
75,97,47,61,53
61,13,29
97,13,75,29,47
The first section specifies the page ordering rules, one per line. The first rule, 47|53, means that if an update includes both page number 47 and page number 53, then page number 47 must be printed at some point before page number 53. (47 doesn't necessarily need to be immediately before 53; other pages are allowed to be between them.)

The second section specifies the page numbers of each update. Because most safety manuals are different, the pages needed in the updates are different too. The first update, 75,47,61,53,29, means that the update consists of page numbers 75, 47, 61, 53, and 29.

To get the printers going as soon as possible, start by identifying which updates are already in the right order.

In the above example, the first update (75,47,61,53,29) is in the right order:

75 is correctly first because there are rules that put each other page after it: 75|47, 75|61, 75|53, and 75|29.
47 is correctly second because 75 must be before it (75|47) and every other page must be after it according to 47|61, 47|53, and 47|29.
61 is correctly in the middle because 75 and 47 are before it (75|61 and 47|61) and 53 and 29 are after it (61|53 and 61|29).
53 is correctly fourth because it is before page number 29 (53|29).
29 is the only page left and so is correctly last.
Because the first update does not include some page numbers, the ordering rules involving those missing page numbers are ignored.

The second and third updates are also in the correct order according to the rules. Like the first update, they also do not include every page number, and so only some of the ordering rules apply - within each update, the ordering rules that involve missing page numbers are not used.

The fourth update, 75,97,47,61,53, is not in the correct order: it would print 75 before 97, which violates the rule 97|75.

The fifth update, 61,13,29, is also not in the correct order, since it breaks the rule 29|13.

The last update, 97,13,75,29,47, is not in the correct order due to breaking several rules.

For some reason, the Elves also need to know the middle page number of each update being printed. Because you are currently only printing the correctly-ordered updates, you will need to find the middle page number of each correctly-ordered update. In the above example, the correctly-ordered updates are:

75,47,61,53,29
97,61,53,29,13
75,29,13
These have middle page numbers of 61, 53, and 29 respectively. Adding these page numbers together gives 143.

Of course, you'll need to be careful: the actual list of page ordering rules is bigger and more complicated than the above example.

Determine which updates are already in the correct order. What do you get if you add up the middle page number from those correctly-ordered updates?
```

In [36]:
import re
import sys
sys.path.append("..")

from common_processing import read_lines, read_as_character_array,\
    read_columns, read_diagonals

test_path = "test.txt"
data_path = "data.txt"

data = read_lines(test_path)
data

['47|53',
 '97|13',
 '97|61',
 '97|47',
 '75|29',
 '61|13',
 '75|53',
 '29|13',
 '97|29',
 '53|29',
 '61|53',
 '97|53',
 '61|29',
 '47|13',
 '75|47',
 '97|75',
 '47|61',
 '75|61',
 '47|29',
 '75|13',
 '53|13',
 '',
 '75,47,61,53,29',
 '97,61,53,29,13',
 '75,29,13',
 '75,97,47,61,53',
 '61,13,29',
 '97,13,75,29,47']

In [37]:
# Split rules from updates:

raw_rules = []
updates = []
add_rules = True
for line in data:
    if line == "":
        add_rules = False
        continue

    if add_rules:
        raw_rules.append(line)
        continue

    updates.append(line.split(","))

raw_rules, updates

(['47|53',
  '97|13',
  '97|61',
  '97|47',
  '75|29',
  '61|13',
  '75|53',
  '29|13',
  '97|29',
  '53|29',
  '61|53',
  '97|53',
  '61|29',
  '47|13',
  '75|47',
  '97|75',
  '47|61',
  '75|61',
  '47|29',
  '75|13',
  '53|13'],
 [['75', '47', '61', '53', '29'],
  ['97', '61', '53', '29', '13'],
  ['75', '29', '13'],
  ['75', '97', '47', '61', '53'],
  ['61', '13', '29'],
  ['97', '13', '75', '29', '47']])

In [38]:
""" 
approach

Update = [x0, x1, x2, ... xn]

rule = xi|xj where xj must always pressent after xi

take an update, check all rules involving x0
does any xn break the update? (is there a rule such that xn|x0 for xn in update[1:]?)

if so, return FALSE (update is not valid)

if not, run the code again with update[1:]

if len(update) == 1, then return TRUE
"""

def parse_rules(raw_rules: list[str]) -> list[tuple]:
    """ 
    create a dictionary with rules.

     NOTE!!! 

    This only works if we have a single rule per key, not the case in dataset

    """
    rule_pairs = [rule.split("|") for rule in raw_rules]

    rules = {}

    for k, v in rule_pairs:
        if k in rules.keys():
            rules[k].append(v)
        else:
            rules[k] = [v]

    return rules

rules = parse_rules(raw_rules)
rules

{'47': ['53', '13', '61', '29'],
 '97': ['13', '61', '47', '29', '53', '75'],
 '75': ['29', '53', '47', '61', '13'],
 '61': ['13', '53', '29'],
 '29': ['13'],
 '53': ['29', '13']}

In [39]:

def check_rules_for(x0, candidates, rules=rules, debug=False) -> bool:
    """ 
    check rules and return wheter x0 is always before all candidates

    return True (valid) as default, change it to False if
    for any candidate xn there is a rull such that xn|x0
    """

    for cand in candidates:
        if cand in rules.keys() and x0 in rules[cand]:
            if debug:
                print("Exlcuded by rule: ",f"{cand}|{x0}")

            return False

    return True

def check_update(update: list, debug=False) -> bool:

    if len(update) == 1:
        if debug:
            print("Update exhausted")
        return True

    x0 = update[0]
    candidates = update[1:]

    result = check_rules_for(x0, candidates,rules=rules)

    if result == True:
        return check_update(candidates)
    else:
        return False
    
valid_updates = []
for update in updates:
    if check_update(update):
        valid_updates.append(update)

print("Valid updates: ", len(valid_updates))
# for vu in valid_updates:
#     print(vu)


Valid updates:  3


In [40]:
# Select the middle element and count them
result = 0

for vu in valid_updates:
    result += int(vu[(len(vu) - 1)//2 ])

print(result)

143


In [41]:
# Try with the whole dataset
data = read_lines(data_path)

raw_rules = []
updates = []
add_rules = True
for line in data:
    if line == "":
        add_rules = False
        continue

    if add_rules:
        raw_rules.append(line)
        continue

    updates.append(line.split(","))

print(len(updates)," Updates")
print(len(raw_rules)," rules")

rules = parse_rules(raw_rules)

valid_updates = []
for update in updates:
    if check_update(update):
        valid_updates.append(update)
print(len(valid_updates)," Valid updates")

# Select the middle element and count them
result = 0

for vu in valid_updates:
    result += int(vu[(len(vu) - 1)//2 ])

print("Sum of the middle elements",result)

192  Updates
1176  rules
85  Valid updates
Sum of the middle elements 4662


# Part Two
```
While the Elves get to work printing the correctly-ordered updates, you have a little time to fix the rest of them.

For each of the incorrectly-ordered updates, use the page ordering rules to put the page numbers in the right order. For the above example, here are the three incorrectly-ordered updates and their correct orderings:

75,97,47,61,53 becomes 97,75,47,61,53.
61,13,29 becomes 61,29,13.
97,13,75,29,47 becomes 97,75,47,29,13.
After taking only the incorrectly-ordered updates and ordering them correctly, their middle page numbers are 47, 29, and 47. Adding these together produces 123.

Find the updates which are not in the correct order. What do you get if you add up the middle page numbers after correctly ordering just those updates?
```


In [42]:
# load test for debugging
data = read_lines(test_path)

raw_rules = []
updates = []
add_rules = True
for line in data:
    if line == "":
        add_rules = False
        continue

    if add_rules:
        raw_rules.append(line)
        continue

    updates.append(line.split(","))

print(len(updates)," Updates")
print(len(raw_rules)," rules")

rules = parse_rules(raw_rules)

6  Updates
21  rules


In [43]:
# Same logic to get desordered updates:

desordered_updates = []
for update in updates:
    if check_update(update) == False:
        desordered_updates.append(update)

print(desordered_updates)



[['75', '97', '47', '61', '53'], ['61', '13', '29'], ['97', '13', '75', '29', '47']]


In [44]:
""" 
How do we order the updates??? 

Get the updates that checked for unordered according to the rules:

if a rule fails, swap x0 and xn, then repeat untill the update is ordered.
"""

def find_broken_rule(x0, candidates, rules=rules, debug=True) -> bool:
    """ 
    check rules and return wheter x0 is always before all candidates

    return True (valid) as default, change it to False if
    for any candidate xn there is a rull such that xn|x0
    """

    for cand in candidates:
        if cand in rules.keys() and x0 in rules[cand]:
            if debug:
                print("Exlcuded by rule: ",f"{cand}|{x0}")
            return cand, x0
            
    return True, True

def correct_update(update) -> list:
    """ 
    Check what rule the update breaks:
    xn|x0
    swap x0 and xn
    check that the update is valid
        if true, return the corrected update
        if not, repeat.
    """

    if check_update(update):
        return update
    
    print(update)
    cand, x0 = find_broken_rule(update[0], candidates=update[1:], rules=rules)

    cand_index, x0_index = update.index(cand), update.index(x0)
    update[cand_index], update[x0_index] =  update[x0_index], update[cand_index]

    return correct_update(update)

correct_update(desordered_updates[0])
    

['75', '97', '47', '61', '53']
Exlcuded by rule:  97|75


['97', '75', '47', '61', '53']

In [45]:
desordered_updates

[['97', '75', '47', '61', '53'],
 ['61', '13', '29'],
 ['97', '13', '75', '29', '47']]

In [46]:
rules

{'47': ['53', '13', '61', '29'],
 '97': ['13', '61', '47', '29', '53', '75'],
 '75': ['29', '53', '47', '61', '13'],
 '61': ['13', '53', '29'],
 '29': ['13'],
 '53': ['29', '13']}

In [53]:
# New approach:
""" 
For every broken update, iterate over it untill it is corrected.
"""

def find_broken_rule(x0, candidates, rules=rules, debug=True) -> bool:
    """ 
    """

    for cand in candidates:
        if cand in rules.keys() and x0 in rules[cand]:
            if debug:
                print("Exlcuded by rule: ",f"{cand}|{x0}")
            return cand, x0
        
    if len(candidates) > 1:
        return find_broken_rule(candidates[0], candidates[1:])

def swap_by_rule(update) -> list:
    x0, candidates = update[0], update[1:]
    try:
        cand, x0 = find_broken_rule(x0, candidates, rules=rules, debug=True)

        # Swap x0 and candidate
        cand_index, x0_index = update.index(cand), update.index(x0)
        update[cand_index], update[x0_index] =  update[x0_index], update[cand_index]

        return update
    
    except Exception as e:
        print(e)
        return update


def correct_update(update: list) -> list:

    if check_update(update):
        return update
    else:
        update = swap_by_rule(update)
        return correct_update(update)

update = desordered_updates[2]
print(find_broken_rule(x0=update[0], candidates=update[1:]))

Exlcuded by rule:  24|33
('24', '33')


In [54]:
corrected_updates =[]
for update in desordered_updates:
    cu = correct_update(update)
    corrected_updates.append(cu)

corrected_updates

Exlcuded by rule:  46|69
Exlcuded by rule:  92|46
Exlcuded by rule:  43|92
Exlcuded by rule:  78|43
Exlcuded by rule:  46|69
Exlcuded by rule:  92|46
Exlcuded by rule:  43|92
Exlcuded by rule:  46|69
Exlcuded by rule:  54|46
Exlcuded by rule:  74|54
Exlcuded by rule:  92|74
Exlcuded by rule:  69|76
Exlcuded by rule:  62|69
Exlcuded by rule:  11|62
Exlcuded by rule:  46|11
Exlcuded by rule:  54|46
Exlcuded by rule:  74|54
Exlcuded by rule:  69|76
Exlcuded by rule:  62|69
Exlcuded by rule:  11|62
Exlcuded by rule:  46|11
Exlcuded by rule:  26|46
Exlcuded by rule:  83|26
Exlcuded by rule:  54|83
Exlcuded by rule:  69|76
Exlcuded by rule:  62|69
Exlcuded by rule:  11|62
Exlcuded by rule:  12|11
Exlcuded by rule:  46|12
Exlcuded by rule:  26|46
Exlcuded by rule:  83|26
Exlcuded by rule:  69|76
Exlcuded by rule:  62|69
Exlcuded by rule:  11|62
Exlcuded by rule:  12|11
Exlcuded by rule:  46|12
Exlcuded by rule:  26|46
Exlcuded by rule:  57|76
Exlcuded by rule:  69|57
Exlcuded by rule:  47|69


[['42',
  '86',
  '78',
  '43',
  '92',
  '74',
  '54',
  '83',
  '26',
  '46',
  '14',
  '12',
  '11',
  '62',
  '85',
  '47',
  '69',
  '57',
  '76',
  '66',
  '48'],
 ['48',
  '15',
  '24',
  '33',
  '41',
  '65',
  '36',
  '38',
  '73',
  '56',
  '19',
  '98',
  '63',
  '77',
  '22',
  '44',
  '75'],
 ['85',
  '57',
  '76',
  '82',
  '48',
  '15',
  '95',
  '24',
  '33',
  '79',
  '41',
  '65',
  '37',
  '38',
  '56',
  '19',
  '98'],
 ['14', '12', '62', '69', '76', '41', '38'],
 ['41',
  '37',
  '38',
  '73',
  '56',
  '19',
  '71',
  '89',
  '63',
  '77',
  '22',
  '72',
  '44',
  '75',
  '23',
  '88',
  '42',
  '86',
  '43'],
 ['78',
  '83',
  '26',
  '46',
  '11',
  '62',
  '47',
  '69',
  '58',
  '76',
  '66',
  '82',
  '48',
  '15',
  '95'],
 ['57', '66', '24', '93', '38', '73', '19', '71', '77'],
 ['41',
  '36',
  '38',
  '73',
  '19',
  '98',
  '89',
  '63',
  '72',
  '75',
  '67',
  '23',
  '42',
  '78',
  '43'],
 ['46', '14', '85', '58', '82', '15', '95', '24', '33', '65'

In [55]:
# Sum the middle element of each corrected update

result = 0

for vu in corrected_updates:
    result += int(vu[(len(vu) - 1)//2 ])

print("Sum of the middle elements", result)

Sum of the middle elements 5900


In [56]:
# load test for debugging
data = read_lines(data_path)

raw_rules = []
updates = []
add_rules = True
for line in data:
    if line == "":
        add_rules = False
        continue

    if add_rules:
        raw_rules.append(line)
        continue

    updates.append(line.split(","))

print(len(updates)," Updates")
print(len(raw_rules)," rules")

rules = parse_rules(raw_rules)

192  Updates
1176  rules


In [51]:
# Same logic to get desordered updates:
desordered_updates = []
for update in updates:
    if check_update(update) == False:
        desordered_updates.append(update)

print("Number of desordered updates: ", len(desordered_updates))

Number of desordered updates:  107


In [57]:
corrected_updates =[]
for update in desordered_updates:
    cu = correct_update(update)
    corrected_updates.append(cu)

In [58]:
result = 0

for vu in corrected_updates:
    result += int(vu[(len(vu) - 1)//2 ])

print("Sum of the middle elements", result)

Sum of the middle elements 5900
